#### Import

In [1]:
import os
import glob
import sys

import shutil
import json

import numpy as np
import random
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import nibabel as nib
from scipy.ndimage import morphology
import SimpleITK
import pandas as pd

### Helper functions 
#### Most of these has already been tested in other notebook(s); So no tester code is added here

In [2]:
#Check folder existence, if not available, create
def checkFolderExistenceAndCreate(folderPath):
    #if folderPath does not exist create it
    if os.path.exists(folderPath):
        #Check if it is a directory or not
        if os.path.isfile(folderPath): 
            sys.exit(folderPath, ' is a file and not directory. Exiting.') 
    else:
        #create 
        os.makedirs(folderPath)

In [3]:
#Read niig.gz file with proper clipping, scaling, normalization if asked for 
def readAndScaleImageData(fileName, folderName, clipFlag, clipLow, clipHigh, scaleFlag, scaleFactor,\
                          meanSDNormalizeFlag, finalDataType, \
                          isLabelData, labels_to_train_list, verbose=False): 
    returnNow = False
    #check file existence
    filePath = os.path.join(folderName, fileName)            
    if os.path.exists(filePath):
        pass
    else:
        print(filePath, ' does not exist')  
        returnNow = True  
    if returnNow:
        sys.exit() 
    #We are here => returnNow = False
    #Also note #axes: depth, height, width
    fileData = np.transpose(nib.load(filePath).get_fdata(), axes=(2,1,0))  
    #Debug code
    if verbose:
        dataMin = fileData.min()
        dataMax = fileData.max()
        print('fileName - shape - type -min -max: ', fileName, ' ', fileData.shape, ' ', fileData.dtype, ' ', dataMin, ' ', dataMax)
    #Clamp                          
    if True == clipFlag:
        np.clip(fileData, clipLow, clipHigh, out= fileData)
    #Scale   
    if True == scaleFlag:
        fileData = fileData / scaleFactor
    #mean SD Normalization
    if True == meanSDNormalizeFlag:
        fileData = (fileData - np.mean(fileData))/np.std(fileData)
    #Type conversion
    fileData = fileData.astype(finalDataType)
    if True == isLabelData:
        # pick specific labels to train (if training labels other than 1s and 0s)
        if labels_to_train_list != [1]:
            temp = np.zeros(shape=fileData.shape, dtype=fileData.dtype)
            new_label_value = 1
            for lbl in labels_to_train_list: 
                ti = (fileData == lbl)
                temp[ti] = new_label_value
                new_label_value += 1
            fileData = temp
    return fileData

In [4]:
#dice functions
def dice_coef_func(a,b):
    a = a.astype(np.uint8).flatten()
    b = b.astype(np.uint8).flatten()
    dice = (2 * np.sum(np.multiply(a,b))) / (np.sum(a) + np.sum(b))
    return dice

def dice_multi_label(test, gt):
    labels = np.unique(gt)
    ti = labels > 0
    unique_lbls = labels[ti]
    dice = np.zeros(len(unique_lbls))
    i = 0
    for lbl_num in unique_lbls:
            ti = (test == lbl_num)
            ti2 = (gt == lbl_num)
            test_mask = np.zeros(test.shape, dtype=np.uint8)
            test_mask[ti] = 1
            gt_mask = np.zeros(gt.shape, dtype=np.uint8)
            gt_mask[ti2] = 1
            dice[i] = dice_coef_func(test_mask, gt_mask)
            i = i + 1
    return dice

In [5]:
# sphere for dilation 
def ball(n):
    struct = np.zeros((2*n+1, 2*n+1, 2*n+1))
    x, y, z = np.indices((2*n+1, 2*n+1, 2*n+1))
    mask = (x - n)**2 + (y - n)**2 + (z - n)**2 <= n**2
    struct[mask] = 1
    return struct.astype(np.bool)

In [6]:
#Get 3D bounding box with extra padding 
#Rewriting: https://stackoverflow.com/questions/31400769/bounding-box-of-numpy-array
def bbox2_3D(img, expandBBoxFlag=False, pad=0):
    """
    In 3D np array assuming:
    - first axis is slice, -> axial plane index
    - 2nd axis row in each slice, -> coronal plane index
    - last axis is column in each slice -> sagittal plane index
    """
    #np.any will search for non zero elemens in the axis mentioned 
    #So if axis=(1,2): Axial plane, it will search over axis (1=row and 2=col) and return  result
    #for each slice - axial plane
    nonZeroAxialPlanes = np.any(img, axis=(1, 2))
    #So if axis=(0,2) - Corronal plane, it will search over axis (0=slice and 2=col) and return  result
    #for corronal plane
    nonZeroCoronalPlanes = np.any(img, axis=(0, 2))
    #So if axis=(0,1)- sagittal plane, it will search over axis (0=slice and 1=row) and return  result
    #for each sagittal plane
    nonZeroSagittalPlanes = np.any(img, axis=(0, 1))
    
    #result from np.any(): [False  True  True  True False]
    #result of applying np.where() : (array([ 1,2,3]),)
    #So its a tuple of 1-D array on which one applies [0][[0, -1]]
    #The first [0] takes the first array element out of the tuple
    #The next [[0,-1]] is using list based indexing and getting the first and last element out

    axial_min, axial_max = np.where(nonZeroAxialPlanes)[0][[0, -1]]
    coronal_min, coronal_max = np.where(nonZeroCoronalPlanes)[0][[0, -1]]
    sagittal_min, sagittal_max = np.where(nonZeroSagittalPlanes)[0][[0, -1]]
    
    if True == expandBBoxFlag:
        axial_min = max(axial_min-pad,0)
        axial_max = min(axial_max+pad,img.shape[0]-1)
        coronal_min = max(coronal_min-pad,0)
        coronal_max = min(coronal_max+pad,img.shape[1]-1)        
        sagittal_min = max(sagittal_min-pad,0)
        sagittal_max = min(sagittal_max+pad,img.shape[2]-1)
    return axial_min, axial_max, coronal_min, coronal_max, sagittal_min, sagittal_max

In [7]:
#Get union of bounding box from two volumes
def getUnionBoundingBoxWithPadding(gt, pred, bbPad):
    """
    gt: volume 1; 1st dim: slice; 2nd dim: row; 3rd dim col
    pred: volume 2 of same shape as gt
    bbPad: Padding amount to be added over union
    
    return: bounding box limits (inclusive on both end)
    """
    #In BB calculation: a : axial, c: corronal, s : sagittal
    #BB around  GT
    a_min_g, a_max_g, c_min_g, c_max_g, s_min_g, s_max_g = bbox2_3D(gt, expandBBoxFlag=False, pad=0)
    #BB around  pred
    a_min_p, a_max_p, c_min_p, c_max_p, s_min_p, s_max_p = bbox2_3D(pred, expandBBoxFlag=False, pad=0)
    #common BB encompassing both GT and pred  and padding added
    a_min, a_max, c_min, c_max, s_min, s_max = \
            min(a_min_g, a_min_p), max(a_max_g, a_max_p),\
            min(c_min_g, c_min_p), max(c_max_g, c_max_p),\
            min(s_min_g, s_min_p), max(s_max_g, s_max_p)   
    #After added padding: Note both GT and Pred has the same shape
    a_min = max(a_min-bbPad,0)
    a_max = min(a_max+bbPad,     gt.shape[0]-1)
    c_min = max(c_min-bbPad,0)
    c_max = min(c_max+bbPad,   gt.shape[1]-1)        
    s_min = max(s_min-bbPad,0)
    s_max = min(s_max+bbPad,  gt.shape[2]-1) 
    return a_min, a_max, c_min, c_max, s_min, s_max

In [8]:
#Choose scribble from misclassified region (3D) : input includes fraction of misclassified pixels 
# to be added as initial scribble as well as scribble-brush diameter
def chooseScribbleFromMissedFGOrWrongCBG3D(misclassifiedRegion, boundingBoxVol, fractionDivider, dilation_diam,\
                                           useAtmostNScribbles):
    """
        misclassifiedRegion : int8 binary volume of fgMissed or bgWrongC with slice as first dimension
        boundingBoxVol:  Binary (int0, 0-1) volume within which scribbles should be limited
        fractionDivider : positive integer by which number of one pixels in a slice will be divide
                to decide what fraction of them will be chosen. 
                If fractionDivider=1, all of them get chosen
        dilation_diam: dimeter of disk : 1,2,3: disk diameter of  scribble
        useAtmostNScribbles: Integer, If <=0, ignored 
    """
    resultBinary = np.zeros_like(misclassifiedRegion)
    #Constrain the misclassified region with boundary volume
    misclassifiedRegion *= boundingBoxVol
    onePixelsThisVol = np.where(misclassifiedRegion)
    onePixelCoordsThisVol = list(zip(onePixelsThisVol[0], onePixelsThisVol[1], onePixelsThisVol[2]))
    #print(onePixelCoordsThisVol)
    numOnePixelCoordsThisVol = len(onePixelCoordsThisVol)
    #Debug:
    #print('numOnePixelCoordsThisVol ', numOnePixelCoordsThisVol)
    numScribblesFromThisVol = numOnePixelCoordsThisVol // fractionDivider
    if int(useAtmostNScribbles) > 0 :
        numScribblesFromThisVol = min(numScribblesFromThisVol, int(useAtmostNScribbles))
    chosenScribbleCoordsThisVol = random.sample(onePixelCoordsThisVol, numScribblesFromThisVol)
    #print(chosenScribbleCoordsThisVol)
    for coord in chosenScribbleCoordsThisVol : resultBinary[coord] = 1
    #dilate in volume       
    #print('result before dilation ')
    #print(resultBinary)
    resultBinary = \
       scipy.ndimage.binary_dilation(resultBinary,structure=ball(dilation_diam)).astype(resultBinary.dtype)
    #print('result after dilation ')
    #print(resultBinary)
    #But make sure it does not go beyond original binary 
    resultBinary = resultBinary * misclassifiedRegion
    #print('result after clipping ')
    #print(resultBinary)
    #Debug
    #print('Debug: numScrVoxelsFromMissed-3D: ', np.sum(resultBinary))
    return resultBinary, numScribblesFromThisVol

In [9]:
#Method to choose scribble in definitely correctly idenified region (3D): 
#Its chosen from a 3D shell within definite region
def chooseScribbleFromDefiniteRegion3D(definiteRegion,  boundingBoxVol,   fractionDivider, dilation_diam,\
                                           useAtmostNScribbles):
    """
        definiteRegion : int8 binary volume of definiteRegion with slice as first dimension 
        boundingBoxVol:  Binary (int0, 0-1) volume within which scribbles should be limited  
        fractionDivider : positive integer by which number of one pixels in a slice will be divide
                to decide what fraction of them will be chosen. 
                If fractionDivider=1, all of them get chosen
        dilation_diam: dimeter of disk : 2, 3, 4 : a diam x diam window is placed to choose scribble
        useAtmostNScribbles: Integer, If <=0, ignored 
    """
    resultBinary = np.zeros_like(definiteRegion)
    #Erode the definite region 
    erodedRegion = \
          scipy.ndimage.binary_erosion(definiteRegion,structure=ball(dilation_diam)).astype(definiteRegion.dtype)
    scribbleShell = definiteRegion - erodedRegion
    #Constrain the scribbleShell region with boundary volume
    scribbleShell *= boundingBoxVol
    
    onePixelsThisVol = np.where(scribbleShell)
    onePixelCoordsThisVol = list(zip(onePixelsThisVol[0], onePixelsThisVol[1], onePixelsThisVol[2]))
    #print(onePixelCoordsThisVol)
    numOnePixelCoordsThisVol = len(onePixelCoordsThisVol)
    #Debug:
    #print('numOnePixelCoordsThisVol ', numOnePixelCoordsThisVol)
    numScribblesFromThisVol = numOnePixelCoordsThisVol // fractionDivider
    if int(useAtmostNScribbles) > 0 :
        numScribblesFromThisVol = min(numScribblesFromThisVol, int(useAtmostNScribbles))
    chosenScribbleCoordsThisVol = random.sample(onePixelCoordsThisVol, numScribblesFromThisVol)
    #print(chosenScribbleCoordsThisVol)
    for coord in chosenScribbleCoordsThisVol : resultBinary[coord] = 1
    #dilate in volume       
    #print('result before dilation ')
    #print(resultBinary)
    resultBinary = \
       scipy.ndimage.binary_dilation(resultBinary,structure=ball(dilation_diam)).astype(resultBinary.dtype)
    #print('result after dilation ')
    #print(resultBinary)
    #But make sure it does not go beyond original binary 
    resultBinary = resultBinary * scribbleShell
    #print('result after clipping ')
    #print(resultBinary)
    #Debug
    #print('Debug: numScrVoxelsFromDefinite-3D: ', np.sum(resultBinary))
    return resultBinary, numScribblesFromThisVol

In [10]:
#function to generate  different scribble regions automatically
def autoGenerateScribbleRegionsAndSeeds3D(gt, pred, bbPad, fractionDivider, dilation_diam,\
                                           useAtmostNScribblesPerRegion):
    """
        gt : int8 binary volume of ground truth  with slice as first dimension 
        pred : int8 binary volume of prediction  with slice as first dimension
        bbPad: padding to be used while creating  BB 
        fractionDivider : positive integer by which number of one pixels in a slice will be divide
                to decide what fraction of them will be chosen. 
                If fractionDivider=1, all of them get chosen
        dilation_diam: dimeter of disk : 2, 3, 4 : a diam x diam window is placed to choose scribble
        useAtmostNScribblesPerRegion: Integer, If <=0, ignored 
        
        Return: 
            binLimit: axial, corronal and sagittal limits of the bounding box to be used in graphcut, 
            bbVolume: binary bounding box volume; 0 out side, 1 inside bounding box, 
            numFGS: number foreground seeds (in missed and definite FG region)
            numBGS: number of backrground seeds  (in wrongly classified and definite BG region) 
            fgScribbleFromFGMissed: foreground scribbles from missed FG region 
            bgScribbleFromBGWrongC: background scribbles from BG region wrongly classified as FG
            fgScribbleFromDefiniteFG: foreground scribbles from definite FG region 
            bgScribbleFromDefiniteBG: BG scribbles from definite BG region
            fgSeeds: union of foreground scribbles / seeds 
            bgSeeds: union of background scribbles / seeds
            seedsForGC: seeds for imcut() : 0: unknown, 1 FG, 2: BG
    """   
    #In BB calculation: a : axial, c: corronal, s : sagittal
    a_min, a_max, c_min, c_max, s_min, s_max = getUnionBoundingBoxWithPadding(gt, pred, bbPad)
    binLimit = [a_min, a_max, c_min, c_max, s_min, s_max]
    bbVolume = np.zeros_like(gt)
    #Note the +1 without which we were notmaking the highest limit 1
    bbVolume[a_min:a_max+1, c_min:c_max+1, s_min:s_max+1]=1

    gt_comp = 1-gt 
    pred_comp = 1-pred
    gtMinusPred = gt * pred_comp 
    predMinusGt = pred * gt_comp

    definiteBG = gt_comp # This is also definite background
    definiteFG = gt * pred #This is definite FG
    fgMissed = gtMinusPred  #  FG missed by classified
    bgWrongC = predMinusGt #  BG wrongly classified as FG

    #misclassifiedRegion, boundingBoxVol, fractionDivider, dilation_diam
    fgScribbleFromFGMissed, numFGSM = chooseScribbleFromMissedFGOrWrongCBG3D(misclassifiedRegion=fgMissed, \
      boundingBoxVol=bbVolume, fractionDivider=fractionDivider,\
    dilation_diam=dilation_diam, useAtmostNScribbles=useAtmostNScribblesPerRegion)
    
    #misclassifiedRegion, boundingBoxVol, fractionDivider, dilation_diam
    bgScribbleFromBGWrongC, numBGSM = chooseScribbleFromMissedFGOrWrongCBG3D(misclassifiedRegion=bgWrongC, \
      boundingBoxVol=bbVolume, fractionDivider=fractionDivider,\
    dilation_diam=dilation_diam, useAtmostNScribbles=useAtmostNScribblesPerRegion)
    
    #definiteRegion,  boundingBox,  dilation_diam
    fgScribbleFromDefiniteFG, numFGSD = chooseScribbleFromDefiniteRegion3D(definiteRegion=definiteFG,\
       boundingBoxVol=bbVolume, fractionDivider=fractionDivider,\
    dilation_diam=dilation_diam, useAtmostNScribbles=useAtmostNScribblesPerRegion)
    
    #definiteRegion,  boundingBox,  dilation_diam
    bgScribbleFromDefiniteBG, numBGSD = chooseScribbleFromDefiniteRegion3D(definiteRegion=definiteBG,\
       boundingBoxVol=bbVolume, fractionDivider=fractionDivider,\
    dilation_diam=dilation_diam, useAtmostNScribbles=useAtmostNScribblesPerRegion)
    
    numFGS = numFGSM + numFGSD
    numBGS = numBGSM + numFGSD
    
    #imCut requires fgSeeds to be 1 and bgSeeds to be 2
    #Convert scribbles into seeds for imcut() 
    # FG and BG seeds  are supposed to be disjoint; Still clip to 1    
    fgSeeds = np.clip(fgScribbleFromFGMissed + fgScribbleFromDefiniteFG, 0, 1)
    bgSeeds = 2 * np.clip(bgScribbleFromBGWrongC + bgScribbleFromDefiniteBG, 0, 1)
    #Again fgSeeds and bgSeeds are supposed to be disjoint. Still clip to 2
    seedsForGC = np.clip(fgSeeds + bgSeeds, 0, 2)    
        
    return binLimit, bbVolume, numFGS, numBGS, fgScribbleFromFGMissed, bgScribbleFromBGWrongC,\
        fgScribbleFromDefiniteFG, bgScribbleFromDefiniteBG, fgSeeds, bgSeeds, seedsForGC

In [11]:
#Configs
patDataConfig = \
{
 'expPatName' : 'expPat',   
 'ctSuffix' :'_ct.nii.gz',
 'ptSuffix': '_pt.nii.gz',
 'gtSuffix': '_ct_gtvt.nii.gz',
 'predSuffix' : '_segment.nii.gz',
 'softmaxSuffix' : '_softMax.nii.gz',
 'fgSeedsSuffix' :'_fgSeeds.nii.gz',
 'bgSeedsSuffix' :'_bgSeeds.nii.gz',
 'gCutSuffix' :'_gCut.nii.gz',   
 'ct_low': -1000,
 'ct_high': 3095,
 'ct_clipImages' : False,
 'ct_scaleImages' : False,
 'ct_scaleFactor' : 1000,
 'ct_normalizeImages' : False,
 'pt_low' : -1000,
 'pt_high' : 3095,
 'pt_clipImages' : False,
 'pt_scaleImages' : False,
 'pt_scaleFactor' : 1,
 'pt_normalizeImages': False,
 'labels_to_train' : [1]
 }

autoScribbleAndGCConfig = \
{
 'bbPad' : 2,
 'fractionDivider' : 10,
 'dilation_diam' : 2,
 'useAtmostNScribblesPerRegion' : 3,
 'segparams_ssgc' : 
 {
  'method': 'graphcut',
  'pairwise_alpha': 1.0,
  'modelparams': 
   {
    'cvtype': 'full',
    'params': {
               'covariance_type': 'full', 
               'n_components': 1
              },
   },
   'return_only_object_with_seeds': True,
 },
 # 'segparams_ssgc2' : 
 # {
 #  'method': 'graphcut',
 #  'use_boundary_penalties': False,
 #  'boundary_dilatation_distance': 2,
 #  'boundary_penalties_weight': 1,
 #  'modelparams': 
 #  {
 #   'type': 'gmmsame',
 #   'params': 
 #    {
 #     'n_components': 2
 #    },
 #    'return_only_object_with_seeds': True,
 #    # 'fv_type': 'fv_extern',
 #    # 'fv_extern': fv_function,
 #    # 'adaptation': 'original_data',
 #  },
 # }     
}

In [12]:
# srcFolder =\
#   'J:/HecktorData/nnUnet_3dfullres/validation_gtvs_withSoftmax'
# expFolder = 'J:/PlayDataManualSegmentation/AutoScribbleExperiment'
srcFolder =\
  '/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/validation_gtvs_withSoftmax'
expFolder = '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment'
listOfPatients = [(os.path.basename(f)).replace('_ct.nii.gz','') \
      for f in glob.glob(srcFolder + '/*_ct.nii.gz', recursive=False) ]
print(listOfPatients)

['CHUM013', 'CHGJ089', 'CHUM055', 'CHGJ048', 'CHGJ057', 'CHUS090', 'CHMR005', 'CHGJ072', 'CHUM062', 'CHGJ088', 'CHUS008', 'CHUM002', 'CHGJ038', 'CHUS053', 'CHUS089', 'CHUS050', 'CHUS021', 'CHUM030', 'CHUS043', 'CHUS047', 'CHMR001', 'CHUS069', 'CHUM042', 'CHMR030', 'CHUM007', 'CHMR016', 'CHGJ053', 'CHUM040', 'CHGJ026', 'CHUS095', 'CHGJ091', 'CHUS016', 'CHUS041', 'CHUM016', 'CHUM061', 'CHUM006', 'CHMR034', 'CHUS086', 'CHUS067', 'CHMR014', 'CHUM050', 'CHUS101', 'CHGJ015', 'CHGJ085', 'CHMR028', 'CHUS073', 'CHUM001', 'CHGJ018', 'CHUS094', 'CHUM021', 'CHUS019', 'CHMR024', 'CHUS056', 'CHUS006', 'CHUM034', 'CHUS005', 'CHUS096', 'CHMR023', 'CHUS057', 'CHUM008', 'CHUS088', 'CHUS085', 'CHUS048', 'CHUM064', 'CHGJ065', 'CHUS020', 'CHUS081', 'CHMR013', 'CHUM051', 'CHUM012', 'CHGJ036', 'CHUM044', 'CHUS049', 'CHGJ086', 'CHGJ046', 'CHUM049', 'CHUM056', 'CHGJ062', 'CHUM063', 'CHUM018', 'CHUM058', 'CHUM027', 'CHUM015', 'CHUM036', 'CHUM053', 'CHUS028', 'CHGJ008', 'CHUS060', 'CHMR040', 'CHUS039', 'CHUM033'

In [13]:
patientName = 'CHUM038'
verbose = True
#check existence of destination folder
checkFolderExistenceAndCreate(expFolder)
#Transfer file into experiment directory with fixed name
shutil.copy(os.path.join(srcFolder, patientName+patDataConfig['ctSuffix']), \
            os.path.join(expFolder, patDataConfig['expPatName']+patDataConfig['ctSuffix']))
shutil.copy(os.path.join(srcFolder, patientName+patDataConfig['ptSuffix']), \
            os.path.join(expFolder, patDataConfig['expPatName']+patDataConfig['ptSuffix']))
shutil.copy(os.path.join(srcFolder, patientName+patDataConfig['gtSuffix']), \
            os.path.join(expFolder, patDataConfig['expPatName']+patDataConfig['gtSuffix']))
shutil.copy(os.path.join(srcFolder, patientName+patDataConfig['predSuffix']), \
            os.path.join(expFolder, patDataConfig['expPatName']+patDataConfig['predSuffix']))
shutil.copy(os.path.join(srcFolder, patientName+patDataConfig['softmaxSuffix']), \
            os.path.join(expFolder, patDataConfig['expPatName']+patDataConfig['softmaxSuffix']))    

#Read np ndarray from the files, to scale or not to scale? 
ctFileName = patDataConfig['expPatName']+patDataConfig['ctSuffix']
ctData = readAndScaleImageData(fileName=ctFileName,\
    folderName=expFolder, clipFlag = patDataConfig['ct_clipImages'],\
    clipLow=patDataConfig['ct_low'], clipHigh =patDataConfig['ct_high'],\
    scaleFlag=patDataConfig['ct_scaleImages'], scaleFactor=patDataConfig['ct_scaleFactor'],\
    meanSDNormalizeFlag = patDataConfig['ct_normalizeImages'], finalDataType = np.float32,\
    isLabelData=False, labels_to_train_list=None, verbose=verbose)

ptFileName = patDataConfig['expPatName']+patDataConfig['ptSuffix']
ptData = readAndScaleImageData(fileName=ptFileName,\
    folderName=expFolder, clipFlag = patDataConfig['pt_clipImages'],\
    clipLow=patDataConfig['pt_low'], clipHigh =patDataConfig['pt_high'],\
    scaleFlag=patDataConfig['pt_scaleImages'], scaleFactor=patDataConfig['pt_scaleFactor'],\
    meanSDNormalizeFlag = patDataConfig['pt_normalizeImages'], finalDataType = np.float32,\
    isLabelData=False, labels_to_train_list=None, verbose=verbose)

gtFileName = patDataConfig['expPatName']+patDataConfig['gtSuffix']
gtData = readAndScaleImageData(fileName=gtFileName,\
    folderName=expFolder, clipFlag = False,\
    clipLow=0, clipHigh = 0,\
    scaleFlag=False, scaleFactor=1, meanSDNormalizeFlag = False, finalDataType = np.int8, \
    isLabelData=True, labels_to_train_list=patDataConfig['labels_to_train'], verbose=verbose)

predFileName = patDataConfig['expPatName']+patDataConfig['predSuffix']
predData = readAndScaleImageData(fileName=predFileName,\
    folderName=expFolder, clipFlag = False,\
    clipLow=0, clipHigh = 0,\
    scaleFlag=False, scaleFactor=1, meanSDNormalizeFlag = False, finalDataType = np.int8, \
    isLabelData=True, labels_to_train_list=patDataConfig['labels_to_train'], verbose=verbose)

softmaxFileName = patDataConfig['expPatName']+patDataConfig['softmaxSuffix']
softmaxData = readAndScaleImageData(fileName=softmaxFileName,\
    folderName=expFolder, clipFlag = False,\
    clipLow=0.0, clipHigh =1.0,\
    scaleFlag=False, scaleFactor=1.0, meanSDNormalizeFlag = False, finalDataType = np.float32,\
    isLabelData=False, labels_to_train_list=None, verbose=verbose)

binLimit, bbVolume, numFGS, numBGS, fgScribbleFromFGMissed, bgScribbleFromBGWrongC,\
    fgScribbleFromDefiniteFG, bgScribbleFromDefiniteBG, fgSeeds, bgSeeds, seedsForGC \
    = autoGenerateScribbleRegionsAndSeeds3D(gt=gtData, pred=predData,\
        bbPad=autoScribbleAndGCConfig['bbPad'],fractionDivider=autoScribbleAndGCConfig['fractionDivider'],\
        dilation_diam=autoScribbleAndGCConfig['dilation_diam'],\
        useAtmostNScribblesPerRegion=autoScribbleAndGCConfig['useAtmostNScribblesPerRegion'])

modelImage_nii_aff = nib.load(os.path.join(expFolder, ctFileName)).affine

#fgScrM_name = expPatName + '_fgScrM_fr_{:>02d}_D_{:>02d}.nii.gz'.format(fractionDivider, dilation_diam)
fgSeedsFileName = patDataConfig['expPatName'] + patDataConfig['fgSeedsSuffix']
fgSeedsFilePath = os.path.join(expFolder, fgSeedsFileName)
nib.save(nib.Nifti1Image(np.transpose(fgSeeds, axes=(2,1,0)),\
                         affine=modelImage_nii_aff),fgSeedsFilePath)

bgSeedsFileName = patDataConfig['expPatName'] + patDataConfig['bgSeedsSuffix']
bgSeedsFilePath = os.path.join(expFolder, bgSeedsFileName)
nib.save(nib.Nifti1Image(np.transpose(bgSeeds, axes=(2,1,0)),\
                         affine=modelImage_nii_aff),bgSeedsFilePath)

fileName - shape - type -min -max:  expPat_ct.nii.gz   (145, 144, 144)   float64   -1438.966064453125   4639.98291015625
fileName - shape - type -min -max:  expPat_pt.nii.gz   (145, 144, 144)   float64   -0.014495441690087318   8.463794708251953
fileName - shape - type -min -max:  expPat_ct_gtvt.nii.gz   (145, 144, 144)   float64   0.0   1.0
fileName - shape - type -min -max:  expPat_segment.nii.gz   (145, 144, 144)   float64   0.0   1.0
fileName - shape - type -min -max:  expPat_softMax.nii.gz   (145, 144, 144)   float64   0.0   1.0


### Interactive scribble and graphcut Plan

In [14]:
import JupyterNotebooksLib as slicernb
slicernb.ViewDisplay('OneUpRedSlice') #'FourUp', 'OneUp3D'
slicer.mrmlScene.Clear()

#ctVolumeNode = slicer.util.addVolumeFromArray(ctData,ijkToRAS=None,name='ctVolume',nodeClassName='vtkMRMLScalarVolumeNode')
ctVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ctFileName))
ptVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ptFileName))
softmaxVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,softmaxFileName))

def createSegmentationNodeFromLabelFile(folderPath, labelFileName, labelMapName, segmentationNodeName, colorTriple, sliceFillFlag):
    #labelMapNode = slicer.util.addVolumeFromArray(labelData,ijkToRAS=None,name=labelMapName,nodeClassName='vtkMRMLScalarVolumeNode')
    labelMapNode = slicer.util.loadLabelVolume(os.path.join(folderPath, labelFileName), properties = {'name': labelMapName})
    segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", segmentationNodeName)
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelMapNode, segmentationNode)
    slicer.mrmlScene.RemoveNode(labelMapNode)
    segmentationNode.GetSegmentation().GetSegment(labelMapName).SetColor(colorTriple[0], colorTriple[1],colorTriple[2])
    if True == sliceFillFlag:
        segmentationNode.GetDisplayNode().SetVisibility2DFill(1)
    else:
        segmentationNode.GetDisplayNode().SetVisibility2DFill(0)
    return segmentationNode

gt_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, gtFileName, \
                         "gtLabelMap", "gtSeg", (0,0,1), False)
pred_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, predFileName, \
                         "predLabelMap", "predSeg", (1,0,0), False)
fgSeeds_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, fgSeedsFileName, \
                         "fgSeedsLabelMap", "fgSeedsSeg", (0.15, 0.7, 0.8), True)
bgSeeds_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, bgSeedsFileName, \
                         "bgSeedsLabelMap", "bgSeedsSeg", (0.8, 0.7, 0.15), True)

# slicer.util.setSliceViewerLayers(background=ctVolumeNode, foreground=ptVolumeNode, label=gt_segmentationNode\
#                                 foregroundOpacity=0.5,labelOpacity=0.5)
slicer.util.setSliceViewerLayers(background=softmaxVolumeNode, foreground=ptVolumeNode)
getNode('expPat_pt').GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLPETProceduralColorNodePET-Heat')

In [15]:
slicer.mrmlScene.Clear()

In [ ]:
#Python interactor output
# >>> expPat_pt_VolumeNode = getNode('expPat_pt')
# >>> type(expPat_pt_VolumeNode)
# <class 'MRMLCorePython.vtkMRMLScalarVolumeNode'>
# >>> expPat_pt_DisplayNode = expPat_pt_VolumeNode.GetDisplayNode()
# >>> type(expPat_pt_DisplayNode)
# <class 'MRMLCorePython.vtkMRMLScalarVolumeDisplayNode'>
# >>> expPat_pt_ColorNode = expPat_pt_DisplayNode.GetColorNode()
# >>> type(expPat_pt_ColorNode)
# <class 'MRMLCorePython.vtkMRMLColorTableNode'>
# >>> expPat_pt_CurrentLUT = expPat_pt_ColorNode.GetLookupTable()
# >>> type(expPat_pt_CurrentLUT)
# <class 'vtkCommonCorePython.vtkLookupTable'>

# >>> indexL = expPat_pt_CurrentLUT.GetIndexedLookup()
# >>> type(indexL)
# <class 'int'>
# >>> print(indexL)
# 0
# >>> expPat_pt_CurrentLUT.SetIndexedLookup(3)
# >>> expPat_pt_CurrentLUT = expPat_pt_ColorNode.GetLookupTable()
# >>> print(expPat_pt_CurrentLUT.GetIndexedLookup())
# 3
# >>> getNode('expPat_pt').GetDisplayNode().GetColorNode().IsTypeOf('vtkMRMLColorTableNode')
# 1
# >>> getNode('expPat_pt').GetDisplayNode().GetColorNode().IsTypeOf('vtkMRMLProceduralColorNode')
# 0

In [35]:
#Experimental read write json file
with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_graphCutInput.json") as fp:
        gcSConfig = json.load(fp)
        fp.close() 
srcFolder = gcSConfig["srcFolder" ]
patDataConfig = gcSConfig["patDataConfig" ]
autoScribbleAndGCConfig = gcSConfig['autoScribbleAndGCConfig']
expFolder = patDataConfig["expFolder"]

print(srcFolder)
print(expFolder)
print(autoScribbleAndGCConfig)
print(patDataConfig['labels_to_train'])

#Make changes

new_expFolder = "/home/user/DMML/Data/PlayDataManualSegmentation/ManualScribble"
new_patDataConfig = patDataConfig
new_patDataConfig["expFolder"]= new_expFolder

new_srcFolder = "/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/training_gtvs_withSoftmax"
new_gcSConfig = gcSConfig
new_gcSConfig["srcFolder" ] = new_srcFolder
new_gcSConfig["patDataConfig" ] = new_patDataConfig
with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/modified_expPat_graphCutInput.json", 'w') as fp:
        json.dump(new_gcSConfig, fp, ) #, indent='' #, indent=4
        fp.close()
        
#Let us test
#Experimental read write json file
with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/modified_expPat_graphCutInput.json") as fp:
        gcSConfig2 = json.load(fp)
        fp.close() 
srcFolder2 = gcSConfig2["srcFolder" ]
patDataConfig2 = gcSConfig2["patDataConfig" ]
autoScribbleAndGCConfig2 = gcSConfig2['autoScribbleAndGCConfig']
expFolder2 = patDataConfig2["expFolder"]

print('####################')
print(srcFolder2)
print(expFolder2)
print(autoScribbleAndGCConfig2)
print(patDataConfig2['labels_to_train'])

/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/validation_gtvs_withSoftmax
/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment
{'bbPad': 2, 'fractionDivider': 10, 'dilation_diam': 2, 'useAtmostNScribblesPerRegion': 3, 'segparams_ssgc': {'method': 'graphcut', 'pairwise_alpha': 1.0, 'modelparams': {'cvtype': 'full', 'params': {'covariance_type': 'full', 'n_components': 1}}, 'return_only_object_with_seeds': True}}
[1]
####################
/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/training_gtvs_withSoftmax
/home/user/DMML/Data/PlayDataManualSegmentation/ManualScribble
{'bbPad': 2, 'fractionDivider': 10, 'dilation_diam': 2, 'useAtmostNScribblesPerRegion': 3, 'segparams_ssgc': {'method': 'graphcut', 'pairwise_alpha': 1.0, 'modelparams': {'cvtype': 'full', 'params': {'covariance_type': 'full', 'n_components': 1}}, 'return_only_object_with_seeds': True}}
[1]
